# Парсинг arXiv

In [22]:
import requests
import sqlite3
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import date

# Получаем данные с arXiv API (последние статьи по computer science)
url = "http://export.arxiv.org/api/query"
params = {
    "search_query": "cat:cs.AI",  # статьи по искусственному интеллекту
    "start": 0,
    "max_results": 5,
    "sortBy": "submittedDate",
    "sortOrder": "descending"
}

response = requests.get(url, params=params)
print("Статус запроса:", response.status_code)

# Парсим XML ответ
root = ET.fromstring(response.content)
namespace = {'atom': 'http://www.w3.org/2005/Atom'}

# Извлекаем данные о статьях и авторах
papers_data = []
authors_data = []
all_categories = set()

paper_id_counter = 1

for entry in root.findall('atom:entry', namespace):
    # Парсим основные данные статьи
    title = entry.find('atom:title', namespace).text.strip().replace('\n', ' ') if entry.find('atom:title', namespace) is not None else "No title"
    summary = entry.find('atom:summary', namespace).text.strip().replace('\n', ' ') if entry.find('atom:summary', namespace) is not None else "No abstract"
    published = entry.find('atom:published', namespace).text[:10] if entry.find('atom:published', namespace) is not None else "2024-01-01"

    # Парсим авторов
    authors = []
    for author in entry.findall('atom:author', namespace):
        author_name = author.find('atom:name', namespace).text if author.find('atom:name', namespace) is not None else "Unknown Author"
        authors.append(author_name)
        authors_data.append((author_name, paper_id_counter))

    # Парсим категории
    categories = []
    for category in entry.findall('atom:category', namespace):
        category_term = category.get('term')
        categories.append(category_term)
        all_categories.add(category_term)

    # Основная категория (первая в списке)
    primary_category = categories[0] if categories else "cs.AI"

    papers_data.append({
        'title': title,
        'abstract': summary,
        'published_date': published,
        'category': primary_category,
        'all_categories': categories,
        'authors': authors
    })

    paper_id_counter += 1

print("Получено статей:", len(papers_data))
print("Уникальных категорий:", len(all_categories))
print("Всего авторов:", len(authors_data))

# Выводим пример данных
print("\nПример статьи:")
first_paper = papers_data[0]
print(f"Заголовок: {first_paper['title'][:80]}...")
print(f"Авторы: {', '.join(first_paper['authors'])}")
print(f"Категории: {', '.join(first_paper['all_categories'])}")

Статус запроса: 200
Получено статей: 5
Уникальных категорий: 7
Всего авторов: 40

Пример статьи:
Заголовок: The Universal Weight Subspace Hypothesis...
Авторы: Prakhar Kaushik, Shravan Chaudhari, Ankit Vaidya, Rama Chellappa, Alan Yuille
Категории: cs.LG, cs.AI, cs.CV


# Создание БД

Спроектируйте структуру таблиц

- Для categories: id (автоинкремент), code (уникальный текст), description (текст)

- Для papers: id, title (обязательный), abstract, published_date, category_id (внешний ключ)

- Для authors: id, name (обязательный), paper_id (внешний ключ)

- Для paper_categories: paper_id, category_id (составной первичный ключ)

Напишите SQL запросы CREATE TABLE

- Используйте правильные типы данных (INTEGER, TEXT, DATE)
- Добавьте ограничения (PRIMARY KEY, FOREIGN KEY, NOT NULL, UNIQUE)

Проверьте связи

- Одна статья принадлежит одной основной категории (один-ко-многим)
- Одна статья может иметь несколько авторов (один-ко-многим)
- Одна статья может принадлежать нескольким категориям (многие-ко-многим)

In [23]:
# Подключаемся к базе данных
conn = sqlite3.connect('arxiv_papers.db')
cursor = conn.cursor()

# TODO 1: Создайте таблицу categories с полями: id, code, description
cursor.execute('''
CREATE TABLE IF NOT EXISTS categories (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    code TEXT NOT NULL UNIQUE,
    description TEXT
)
''')

# TODO 2: Создайте таблицу papers с полями: id, title, abstract, published_date, category_id
# Добавьте внешний ключ на таблицу categories
cursor.execute('''
CREATE TABLE IF NOT EXISTS papers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    abstract TEXT,
    published_date DATE,
    category_id INTEGER,
    FOREIGN KEY (category_id) REFERENCES categories (id)
)
''')

# TODO 3: Создайте таблицу authors с полями: id, name, paper_id
# Добавьте внешний ключ на таблицу papers
cursor.execute('''
CREATE TABLE IF NOT EXISTS authors (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    paper_id INTEGER,
    FOREIGN KEY (paper_id) REFERENCES papers (id)
)
''')

# TODO 4: Создайте таблицу paper_categories для связи многие-ко-многим
# Должна содержать paper_id, category_id и составной первичный ключ
cursor.execute('''
CREATE TABLE IF NOT EXISTS paper_categories (
    paper_id INTEGER,
    category_id INTEGER,
    PRIMARY KEY (paper_id, category_id),
    FOREIGN KEY (paper_id) REFERENCES papers (id),
    FOREIGN KEY (category_id) REFERENCES categories (id)
)
''')

conn.commit()

Подготовка данных категорий

- Создайте список кортежей из уникальных категорий
- Для описания используйте формат "Category: {code}"

Вставка категорий

- Напишите SQL запрос INSERT для таблицы categories
- Используйте "OR IGNORE" чтобы пропустить существующие категории
- Преобразуйте результат SELECT в словарь {код_категории: id}

Вставка статей и связей

- Напишите INSERT запрос для таблицы papers
- Используйте cursor.lastrowid для получения ID новой статьи
- Напишите INSERT запрос для таблицы paper_categories

Вставка авторов

- Напишите INSERT запрос для таблицы authors
- Убедитесь, что данные authors_data соответствуют структуре запроса

Подсказки:
- Используйте параметризованные запросы с ?
- Для category_mapping используйте словарь comprehension
- Проверяйте наличие категории перед вставкой в paper_categories

In [24]:
# TODO 1: Подготовьте данные категорий для вставки
# Создайте список кортежей (code, description) для всех уникальных категорий

categories_data = [(code, f'Category: {code}') for code in all_categories]

# TODO 2: Вставьте категории в таблицу categories
# Используйте executemany и INSERT OR IGNORE чтобы избежать дубликатов
cursor.executemany(
    "INSERT OR IGNORE INTO categories (code, description) VALUES (?, ?)",
    categories_data
)

# TODO 3: Создайте словарь для сопоставления кода категории с её ID
cursor.execute("SELECT id, code FROM categories")
rows = cursor.fetchall()
category_mapping = {row[1]: row[0] for row in rows}

# TODO 4: Вставьте статьи в таблицу papers
for i, paper in enumerate(papers_data, 1):
    # Получите category_id для текущей статьи
    category_id = category_mapping.get(paper['category'])

    # Вставьте данные статьи
    cursor.execute(
        "INSERT OR IGNORE INTO papers (title, abstract, published_date, category_id) VALUES (?, ?, ?, ?)",
        (paper['title'], paper['abstract'], paper['published_date'], category_id)
    )

    # Получите ID только что вставленной статьи
    paper_id = cursor.lastrowid

    # TODO 5: Добавьте связи статьи со всеми её категориями
    for category_code in paper['all_categories']:
        cat_id = category_mapping.get(category_code)
        if cat_id:
            cursor.execute(
                "INSERT OR IGNORE INTO paper_categories (paper_id, category_id) VALUES (?, ?)",
                (paper_id, cat_id)
            )

# TODO 6: Вставьте авторов в таблицу authors
cursor.executemany(
    "INSERT OR IGNORE INTO authors (name, paper_id) VALUES (?, ?)",
    authors_data
)

conn.commit()
print(f"Добавлено: {len(papers_data)} статей, {len(authors_data)} авторов, {len(all_categories)} категорий")

Добавлено: 5 статей, 40 авторов, 7 категорий


## Визуализация pandas dataframe

In [25]:
# papers
pd.read_sql_query("SELECT * FROM papers", conn, index_col='id').head()

,title,abstract,published_date,category_id
id,,,,
1,The Universal Weight Subspace Hypothesis,We show that deep neural networks trained acro...,2025-12-04,4
2,DraCo: Draft as CoT for Text-to-Image Preview ...,Recent unified multimodal large language model...,2025-12-04,7
3,ShadowDraw: From Any Object to Shadow-Drawing ...,"We introduce ShadowDraw, a framework that tran...",2025-12-04,7
4,Semantic Soft Bootstrapping: Long Context Reas...,Long context reasoning in large language model...,2025-12-04,2
5,TV2TV: A Unified Framework for Interleaved Lan...,"Video generation models are rapidly advancing,...",2025-12-04,4


In [26]:
# authors
pd.read_sql_query("SELECT * FROM authors", conn, index_col='id').head()

,name,paper_id
id,,
1,Prakhar Kaushik,1
2,Shravan Chaudhari,1
3,Ankit Vaidya,1
4,Rama Chellappa,1
5,Alan Yuille,1


In [27]:
# categories
pd.read_sql_query("SELECT * FROM categories", conn, index_col='id').head()

,code,description
id,,
1,cs.GR,Category: cs.GR
2,cs.CL,Category: cs.CL
3,eess.SP,Category: eess.SP
4,cs.LG,Category: cs.LG
5,cs.AI,Category: cs.AI


In [28]:
# paper_categories
pd.read_sql_query("SELECT * FROM paper_categories", conn).head()

,paper_id,category_id
0,1,4
1,1,5
2,1,7
3,2,7
4,2,5


# Обновление БД

CREATE - Создание новой статьи

- Напишите INSERT запрос для таблицы papers
- Используйте модифицированные данные из существующей статьи
- Получите ID новой статьи с помощью cursor.lastrowid

READ - Выборка категорий

- Напишите SELECT запрос для получения всех полей из categories
- Ограничьте результат 5 записями

READ - Сложный JOIN запрос

- Объедините таблицы papers, categories и authors
- Используйте GROUP_CONCAT для объединения имен авторов
- Сгруппируйте результаты по ID статьи

UPDATE - Обновление данных

- Напишите UPDATE запрос для изменения аннотации
- Используйте WHERE для указания конкретной статьи

DELETE - Удаление данных

- Напишите DELETE запрос для удаления тестовых статей
- Используйте LIKE для поиска по шаблону в заголовке

Подсказки:

- Для JOIN используйте INNER JOIN между связанными таблицами
- Для GROUP_CONCAT укажите разделитель ', '
- В UPDATE обязательно используйте WHERE чтобы не обновить все записи
- В DELETE используйте условие для выбора только тестовых данных

In [29]:
# TODO 1: CREATE - Добавьте новую статью
# Используйте данные из papers_data[0] как шаблон, но измените заголовок и аннотацию
new_paper_data = papers_data[0]

# извлекаем category_id
cursor.execute("SELECT id FROM categories WHERE code = ?", (new_paper_data['category'],))
new_paper_cat_id = cursor.fetchone()[0]

cursor.execute(
    "INSERT INTO papers (title, abstract, published_date, category_id) VALUES (?, ?, ?, ?)",
    (f"ИССЛЕДОВАНИЕ: {new_paper_data['title']}",
     new_paper_data['abstract'][:100],
     date.today().isoformat(),
     new_paper_cat_id)
    # заголовок с префиксом "ИССЛЕДОВАНИЕ:", обрезанная аннотация, текущая дата, category_id
)

new_paper_id = cursor.lastrowid

# TODO 2: READ - Выберите первые 5 категорий из базы
cursor.execute("SELECT * FROM categories LIMIT 5")
categories_list = cursor.fetchall()
print("Категории в базе:")
for row in categories_list:
    print(f"ID: {row[0]}, Код: {row[1]}, Описание: {row[2]}")

# TODO 3: READ - Напишите JOIN запрос для получения статей с их авторами
# Объедините таблицы papers, categories и authors
# Сгруппируйте по статье и объедините имена авторов через запятую
cursor.execute('''
SELECT p.title, c.code, GROUP_CONCAT(a.name, ', ')
FROM papers p
JOIN categories c ON p.category_id = c.id
LEFT JOIN authors a ON p.id = a.paper_id
GROUP BY p.id
ORDER BY p.published_date DESC
LIMIT 5
''')

results = cursor.fetchall()
print("\nСтатьи с авторами:")
for title, category, authors in results:
    print(f"Категория: {category}")
    print(f"Заголовок: {title[:60]}...")
    print(f"Авторы: {authors}")
    print("---")

# TODO 4: UPDATE - Обновите аннотацию второй статьи
cursor.execute(
    "UPDATE papers SET abstract = ? WHERE id = ?",
    (f"ОБНОВЛЕННАЯ АННОТАЦИЯ: {papers_data[1]['abstract'][:100]}", 2)
    # новая аннотация с префиксом "ОБНОВЛЕННАЯ АННОТАЦИЯ:" и ID статьи
)

# TODO 5: DELETE - Удалите тестовые статьи, созданные в этом блоке
cursor.execute("DELETE FROM papers WHERE title LIKE 'ИССЛЕДОВАНИЕ:%'")

conn.commit()

Категории в базе:
ID: 1, Код: cs.GR, Описание: Category: cs.GR
ID: 2, Код: cs.CL, Описание: Category: cs.CL
ID: 3, Код: eess.SP, Описание: Category: eess.SP
ID: 4, Код: cs.LG, Описание: Category: cs.LG
ID: 5, Код: cs.AI, Описание: Category: cs.AI

Статьи с авторами:
Категория: cs.LG
Заголовок: ИССЛЕДОВАНИЕ: The Universal Weight Subspace Hypothesis...
Авторы: None
---
Категория: cs.LG
Заголовок: The Universal Weight Subspace Hypothesis...
Авторы: Alan Yuille, Ankit Vaidya, Prakhar Kaushik, Rama Chellappa, Shravan Chaudhari
---
Категория: cs.CV
Заголовок: DraCo: Draft as CoT for Text-to-Image Preview and Rare Conce...
Авторы: Claire Guo, Dongzhi Jiang, Haodong Li, Hongsheng Li, Jun He, Junyan Ye, Renrui Zhang, Rongyao Fang, Rui Liu, Weijia Li, Zhuofan Zong, Ziyu Guo
---
Категория: cs.CV
Заголовок: ShadowDraw: From Any Object to Shadow-Drawing Compositional ...
Авторы: Noah Snavely, Rundong Luo, Wei-Chiu Ma
---
Категория: cs.CL
Заголовок: Semantic Soft Bootstrapping: Long Context Reasonin

# Анализ и сохранение БД

SQL запрос для основного DataFrame

- Напишите SELECT с JOIN между papers и categories
- Добавьте подзапрос для подсчета количества авторов каждой статьи
- Отсортируйте по дате публикации (DESC)

Создание DataFrame

- Используйте cursor.fetchall() для получения данных
- Задайте понятные названия колонок на русском языке

Анализ категорий

- Используйте метод value_counts() для анализа распределения по категориям

SQL запрос для авторов

- Напишите JOIN запрос между authors, papers и categories
- Ограничьте результат 10 записями для предварительного просмотра

Анализ количества авторов

- Используйте value_counts() с сортировкой по индексу
- Проанализируйте, сколько авторов обычно у статей

Сохранение данных

- Сохраните DataFrame в CSV файл без индекса
- Укажите кодировку utf-8 для корректного сохранения символов

Подсказки:

- Для подсчета авторов используйте подзапрос: (SELECT COUNT(*) FROM authors WHERE paper_id = p.id)
- Для сортировки по убыванию используйте ORDER BY column DESC
- Для ограничения результатов используйте LIMIT
- Для сохранения CSV используйте метод to_csv()

In [30]:
# TODO 1: Создайте SQL запрос для получения данных статей с количеством авторов
# Выберите: id, title, abstract, published_date, category code, количество авторов
# Отсортируйте по дате публикации (новые сначала)
cursor.execute('''
SELECT p.id, p.title, p.abstract, p.published_date, c.code,
       (SELECT COUNT(*) FROM authors WHERE paper_id = p.id) as author_count
FROM papers p
JOIN categories c ON p.category_id = c.id
ORDER BY p.published_date DESC
''')

# TODO 2: Создайте DataFrame из результатов запроса
# Укажите названия колонок: ['ID', 'Название', 'Аннотация', 'Дата', 'Категория', 'Кол-во_авторов']
df = pd.DataFrame(cursor.fetchall(),
                 columns=['ID', 'Название', 'Аннотация', 'Дата', 'Категория', 'Кол-во_авторов'])
print("DataFrame со статьями:")
print(df[['ID', 'Категория', 'Дата', 'Кол-во_авторов']])
print(f"\nВсего статей в DataFrame: {len(df)}")

# TODO 3: Проанализируйте распределение статей по категориям
print("\nСтатистика по категориям:")
category_stats = df['Категория'].value_counts()  # посчитайте количество статей в каждой категории
print(category_stats)

# TODO 4: Создайте DataFrame с информацией об авторах
# Выберите: имя автора, название статьи, категорию
# Ограничьте результат 10 записями
cursor.execute('''
SELECT a.name, p.title, c.code
FROM authors a
JOIN papers p ON a.paper_id = p.id
JOIN categories c ON p.category_id = c.id
LIMIT 10
''')

authors_df = pd.DataFrame(cursor.fetchall(), columns=['Автор', 'Статья', 'Категория'])
print("\nDataFrame с авторами (первые 10 записей):")
print(authors_df)

# TODO 5: Проанализируйте распределение количества авторов по статьям
print("\nСтатистика по авторам:")
author_stats = df['Кол-во_авторов'].value_counts().sort_index()  # посчитайте, сколько статей имеет 1, 2, 3 и т.д. авторов
print("Количество авторов на статью:")
for count, freq in author_stats.items():
    print(f"{count} авторов: {freq} статей")

# TODO 6: Сохраните основной DataFrame в CSV файл
df.to_csv('arxiv_papers.csv', index=False, encoding='utf-8')
print("\nDataFrame сохранен в arxiv_papers.csv")

DataFrame со статьями:
   ID Категория        Дата  Кол-во_авторов
0   1     cs.LG  2025-12-04               5
1   2     cs.CV  2025-12-04              12
2   3     cs.CV  2025-12-04               3
3   4     cs.CL  2025-12-04               2
4   5     cs.LG  2025-12-04              18

Всего статей в DataFrame: 5

Статистика по категориям:
Категория
cs.LG    2
cs.CV    2
cs.CL    1
Name: count, dtype: int64

DataFrame с авторами (первые 10 записей):
               Автор                                             Статья  \
0    Prakhar Kaushik           The Universal Weight Subspace Hypothesis   
1  Shravan Chaudhari           The Universal Weight Subspace Hypothesis   
2       Ankit Vaidya           The Universal Weight Subspace Hypothesis   
3     Rama Chellappa           The Universal Weight Subspace Hypothesis   
4        Alan Yuille           The Universal Weight Subspace Hypothesis   
5      Dongzhi Jiang  DraCo: Draft as CoT for Text-to-Image Preview ...   
6       Renrui Zhang